In [ ]:
import os
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.io import WriteToText
import re

table_spec = '`bigquery-public-data.hacker_news.comments`'

options=PipelineOptions()

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/jupyter-anurag/service3.json'
print(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])

options.view_as(SetupOptions).save_main_session = True
options.view_as(GoogleCloudOptions).project = "anurag-project-1"
options.view_as(StandardOptions).runner = 'DataflowRunner'
options.view_as(GoogleCloudOptions).temp_location='gs://anurag-manila1/temp'
options.view_as(GoogleCloudOptions).staging_location='gs://anurag-manila1/staging'


query = 'SELECT text FROM ' + table_spec + ';'

def CountBiGrams(text):
    if text is None:
        return []
    lt = text.lower()
    words = re.split("\\W+", lt)
    return [ (words[i] + " " + words[i + 1], 1) for i in range(len(words) - 1)]  

print(query)
with beam.Pipeline(options=options) as p:
    result = (
    p
    | 'ReadTable' >> beam.io.Read(beam.io.BigQuerySource(
        query=query, use_standard_sql=True))
    | beam.Map(lambda elem: elem['text'])
        | 'bigram' >> beam.FlatMap(CountBiGrams)
        | beam.CombinePerKey(sum)
        | beam.Filter(lambda wc: wc[1] > 20)
        | WriteToText('gs://anurag-manila1/output/bigrams')
    )


/home/jupyter-anurag/service3.json
SELECT text FROM `bigquery-public-data.hacker_news.comments`;


In [13]:
import os
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions

options=PipelineOptions()

class SquareFn(beam.DoFn):
    def start_bundle(self):
        # Do a one time operation
        pass

    def process(self, x):
        return [x * x];

class AverageFn(beam.CombineFn):
  def create_accumulator(self):
    return []  # sum, count

  def add_input(self, accumulated_value, input):
    accumulated_value.append(input)
    return accumulated_value;

  def merge_accumulators(self, accumulators):
    result = []
    for acc in accumulators:
        result.extend(acc)
    return result

  def extract_output(self, sum_count):
    if len(sum_count) == 0:
        return 0
    return sum(sum_count) / len(sum_count);


with beam.Pipeline(options=options) as p:
    result = (
    p
    | beam.Create([3, 2, 1, 5, 4])
        | "test" >> beam.ParDo(SquareFn())
        | beam.CombineGlobally(AverageFn())
        | beam.ParDo(lambda x: print(x))
    )


9.166666666666666
